In [1]:
import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci

In [2]:
import numpy as np
import sys
sys.path.append( './src/' )

In [3]:
matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)

In [4]:
# H    0.7493682,0.0000000,0.4424329
# O    0.0000000,0.0000000,-0.1653507
# H   -0.7493682,0.0000000,0.4424329

# Define Molecule 

- Note first N atoms will be the ACTIVE portion of the molecule
- the remaining atoms will be the environment

In [5]:
geometry = [
['H@1', (0.7493682,0.0000000,0.4424329)],
['O@1', (0.0000000,0.0000000,-0.1653507)],
    
['H#', (-0.7493682,0.0000000,0.4424329)]
]

# geometry = [
# ['Be@1', (0.0000000,0.0000000,0.0000000)],
# ['H@1', (0.0000000,0.0000000,+1)],
    
# ['H#', (0.0000000,0.0000000,-1)]
# ]

# geometry = [
# ['Li', (0.0000000,0.0000000,0.0000000)],
# ['H', (0.0000000,0.0000000,+1.54)],
    
# ]

In [6]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(*geometry[0][1:][0], marker='o', color='green', label='systemA')
ax.scatter(*geometry[1][1:][0], marker='o', color='green', label='systemA')

for x in range(2, len(geometry)):
    ax.scatter(*geometry[x][1:][0], marker='o', color='red', label='systemB')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

# Define Params

- what level to treat active and envirnoment systems

In [7]:
## define params

basis =  'STO-3G' # '6-31g' #
low_level_xc_functional_or_HF = 'lda, vwn' #'hf'
high_level_xc_functional = 'b3lyp'

low_level_method = 'rhf'
high_level_ref = 'rhf'
high_level_method = 'CCSD'

# Build the GLOBAL (full) molecule

In [8]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()
full_system_mol.atom

[['H@1', (0.7493682, 0.0, 0.4424329)],
 ['O@1', (0.0, 0.0, -0.1653507)],
 ['H#', (-0.7493682, 0.0, 0.4424329)]]

In [9]:
full_system_mol.verbose = 1
full_system_mol.max_memory = 8_000 #MB

# Run Supersystem Calculation (cheap method)

In [10]:
full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional_or_HF
full_system_scf.kernel()

-74.73474041595249

In [11]:
full_system_scf.e_tot

-74.73474041595249

In [12]:
full_system_scf.conv_check

True

In [13]:
two_e_term_total = full_system_scf.get_veff()
e_xc_total = two_e_term_total.exc
v_xc_total = two_e_term_total - full_system_scf.get_j() 

In [14]:
# full_system_scf.get_fock()
# full_system_scf.get_hcore()

In [15]:
# full_system_scf.mo_coeff

# Run expensive CI calc to compare all results too

In [16]:
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= 8_000
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -75.015352478525


# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}$$

- in an orthognal basis $S$ should be the identity matrix (as it is overlap of matrix of orthogonal orbs), SO WE GET:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu} = \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

In [17]:
S_mat = full_system_scf.get_ovlp()
print(np.around(S_mat))

[[ 1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0. -0.  0.]
 [ 0.  0.  0.  1.  0.  0. -0.]
 [ 0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -0.  0.  0.  1.  0.]
 [ 0.  0.  0. -0.  0.  0.  1.]]


- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

In [18]:
from pyscf import lo

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

In [19]:
# Take occupied orbitals of global system calc
occ_orbs = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]

# run localization scheme
PM = lo.PipekMezey(full_system_mol, occ_orbs)
PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
C_mull = PM.kernel()

C_loc_occ = PM.mo_coeff # <-- these are the NEW C matrix (where orbitals are localized to atoms)

In [20]:
np.array_equal(C_mull, C_loc_occ)

True

In [21]:
## check number of electrons is still the same 
# should be the same after change of basis has been done

dm_localised = 2* C_loc_occ @ C_loc_occ.conj().T
S = full_system_scf.get_ovlp()

print(np.isclose(np.trace(dm_localised@S), full_system_mol.nelectron))

True


# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**
    
    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment

In [22]:
## pyscf notes

# order of atoms in geometry is important
full_system_mol.atom

[['H@1', (0.7493682, 0.0, 0.4424329)],
 ['O@1', (0.0, 0.0, -0.1653507)],
 ['H#', (-0.7493682, 0.0, 0.4424329)]]

In [23]:
# aoslice_by_atom gives a matrix:
full_system_mol.aoslice_by_atom()

# where rows are defined by atoms (given by order in full_system_mol.atom - see cell above)
# the 3rd column gives first basis function index for THAT atom
# the last(4th) column gives the last basis function index for THAT atom

array([[0, 1, 0, 1],
       [1, 4, 1, 6],
       [4, 5, 6, 7]])

Therefore loop over basis function centered on the active atoms and find mulliken charge // population

In [24]:
# https://pubs.acs.org/doi/pdf/10.1021/ct300544e
# paper states there are 5 active orbs... where mulliken_population threshold set to 0.4
# here I get the result if threshold set to 0.9

# NOTE in molecule the active atoms must be listed FIRST!

THRESHOLD = 1.5
AO_slice_matrix = full_system_mol.aoslice_by_atom()
N_active_atoms = 2

active_ao_ind=[]

PS_matrix = dm_localised @ S
atomic_charges_list = full_system_mol.atom_charges()
for atm_i, atm_str in enumerate([atom_str for atom_str, _ in full_system_mol.atom][:N_active_atoms]):
    slice_ind = list(range(AO_slice_matrix[atm_i, 2], AO_slice_matrix[atm_i, 3]))
    
    print(atm_str)
    qA = atomic_charges_list[atm_i] #atomic charge of atom_i
    for mu in slice_ind:
        mulliken_charge = qA - PS_matrix[mu,mu]  # mulliken charge of ao centred on atom_i
        mulliken_population = PS_matrix[mu,mu]  # mulliken pop of ao centred on atom_i
#         print(mulliken_charge)
        print(mulliken_population)
        if mulliken_population>THRESHOLD: ### <--- not sure about this THRESHOLD
            active_ao_ind.append(mu)
        
    print('###')

active_ao_ind

H@1
0.8132018340805993
###
O@1
1.9969765302753064
1.857841048267
1.0908988035870821
2.0
1.427879949709412
###


[1, 2, 4]

In [25]:
# assign the rest of the missing indices to the environment

enviro_ao_ind = [ind for ind in range(C_loc_occ.shape[1]) if ind not in active_ao_ind]
enviro_ao_ind

[0, 3]

In [26]:
full_system_mol.aoslice_nr_by_atom()[1][3]

6

## METHOD 2 - SPADE

In [27]:
###### SPADE

# Take occupied orbitals of global system calc
occ_orbs = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]


N_active_AO = AO_slice_matrix[N_active_atoms-1][3]  # find max AO index for active atoms (neg 1 as python indexs from 0)

orthogonal_orbitals = occ_orbs[:N_active_AO, :] # take active rows and all columns

u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)

# find where largest step change 
delta_s = [(i, (singular_values[i] - singular_values[i+1] )) for i in range(len(singular_values) - 1)] # contains (index, delta_s)
n_act_mos = max(delta_s, key=lambda x: x[0])[0] + 1 # finds index where largest step change is! (adds 1 for python indexing)
n_env_mos = len(singular_values) - n_act_mos

###
# next rotate orbtials using SVD rotation - selecting active and enironment portions

In [28]:
# localization = 'SPADE'
localization = 'PM_loc'

if localization == 'SPADE':
    
    # next rotate occupied orbitals of global system calc (NOT localized)
    # using SVD rotation - selecting active and enironment portions
    
    act_orbitals = occ_orbs @ rotation_matrix.T[:, :n_act_mos]
    env_orbitals = occ_orbs @ rotation_matrix.T[:, n_act_mos:]
    
    GLOBAL_rotated_orbitals = occ_orbs @ rotation_matrix.T
    
    dm_active =  2 * act_orbitals @ act_orbitals.T
    dm_enviro =  2 * env_orbitals @ env_orbitals.T
    
    active_ao_ind = list(range(n_act_mos))
    enviro_ao_ind = list(range(n_act_mos, len(singular_values)))
    
elif  localization == 'PM_loc':
    
    # take active and envirnoment orbs from LOCALIZED ORBITALS
    act_orbitals = C_loc_occ[:, active_ao_ind]
    env_orbitals = C_loc_occ[:, enviro_ao_ind]
    
    GLOBAL_rotated_orbitals = C_loc_occ
    
    n_act_mos = len(active_ao_ind)
    n_env_mos = len(enviro_ao_ind) # full_system_mol.aoslice_by_atom()[-1,-1] - n_act_mos

    dm_active =  2 * act_orbitals @ act_orbitals.T
    dm_enviro =  2 * env_orbitals @ env_orbitals.T
    
print(f'number of e- in system = {full_system_mol.nelectron}')
print('number of e- in subsystems:', np.trace(dm_active@S) + np.trace(dm_enviro@S))

number of e- in system = 10
number of e- in subsystems: 9.999999999999996


In [29]:
# checking denisty matrix parition makes sense:

# gamma_rotated_FULL = gamma_act_rotated + gamma_env_rotated
dm_act_and_env = 2* GLOBAL_rotated_orbitals@ GLOBAL_rotated_orbitals.conj().T
np.allclose(dm_active+dm_enviro, dm_act_and_env)

True

# Save orbitals to view

Save localized orbitals as molden file

(allowing MOs to be viewed, to see if localization makes sense)

open with: https://avogadro.cc/

In [30]:
with open('LOCALIZED_orbs.molden', 'w') as outfile:
    tools.molden.header(full_system_mol, outfile)
    tools.molden.orbital_coeff(
                        full_system_mol,
                         outfile, 
                         act_orbitals, # <- active orbitals!
                         ene=full_system_scf.mo_energy[np.array(active_ao_ind)],
                         occ=full_system_scf.mo_occ[np.array(active_ao_ind)])

In [31]:
########################## mulliken pop check START

In [32]:
# https://github.com/pyscf/pyscf/blob/master/pyscf/scf/hf.py

Mulliken_population = np.einsum('ij,ji->i', dm_localised, S).real
Mulliken_population

array([0.81320183, 1.99697653, 1.85784105, 1.0908988 , 2.        ,
       1.42787995, 0.81320183])

In [33]:
mulliken_charges = np.zeros(full_system_mol.natm)
for i, s in enumerate(full_system_mol.ao_labels(fmt=None)):
    mulliken_charges[s[0]] += Mulliken_population[i]
mulliken_charges = full_system_mol.atom_charges() - mulliken_charges
mulliken_charges

array([ 0.18679817, -0.37359633,  0.18679817])

In [34]:
full_system_mol.mulliken_pop(dm=dm_localised)

 ** Mulliken pop  **
pop of  0 H@1 1s        0.81320
pop of  1 O@1 1s        1.99698
pop of  1 O@1 2s        1.85784
pop of  1 O@1 2px       1.09090
pop of  1 O@1 2py       2.00000
pop of  1 O@1 2pz       1.42788
pop of  2 H# 1s        0.81320
 ** Mulliken atomic charges  **
charge of  0H@1 =      0.18680
charge of  1O@1 =     -0.37360
charge of  2H# =      0.18680


(array([0.81320183, 1.99697653, 1.85784105, 1.0908988 , 2.        ,
        1.42787995, 0.81320183]),
 array([ 0.18679817, -0.37359633,  0.18679817]))

In [35]:
########################z## mulliken pop check END

# Embedding

In [36]:
#It seems that PySCF lumps J and K in the J array 
J_act = full_system_scf.get_j(dm = dm_active)
K_act = np.zeros_like(J_act)
two_e_term_act =  full_system_scf.get_veff(dm=dm_active)
e_xc_act = two_e_term_act.exc
v_xc_act = two_e_term_act - J_act 

E_act = np.einsum('ij, ij', dm_active, full_system_scf.get_hcore() + J_act/2) + e_xc_act
E_act

-32.1153079521456

In [37]:
full_system_scf.energy_elec(dm=dm_active)[0]

-32.115307952145585

In [38]:
#It seems that PySCF lumps J and K in the J array 
J_env = full_system_scf.get_j(dm = dm_enviro)
K_env = np.zeros_like(J_env)
two_e_term_env =  full_system_scf.get_veff(dm=dm_enviro)
e_xc_env = two_e_term_env.exc
v_xc_env = two_e_term_env - J_env 

E_env = np.einsum('ij, ij', dm_enviro, full_system_scf.get_hcore() + J_env/2) + e_xc_env
E_env

-70.30417756950928

In [39]:
full_system_scf.energy_elec(dm=dm_enviro)[0]

-70.30417756950924

In [40]:
j_cross = 0.5 * (matrix_dot(dm_active, J_env) + matrix_dot(dm_enviro, J_act))
k_cross = 0.0

In [41]:
xc_cross = e_xc_total - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross

# Define projection method

In [42]:
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_act_and_env)
    F_gammaB_S = Fock @ dm_enviro @ S
    projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector = mu * (S @ dm_enviro  @ S)
else:
    raise ValueError(f'Unknown projection method {projector_method}')



# Define the embbed matrix

$$h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$$

$$v^{\text{embed}} = g[\gamma^{A_{\text{act}}} + \gamma^{B_{\text{env}}} ] - g[\gamma^{A_{\text{act}}}] + P_{\text{projector}}$$

In [43]:
# define the embedded term
g_A_and_B = full_system_scf.get_veff(dm=dm_active+dm_enviro)

g_A = full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector

In [44]:
# PsiEmbed definition
v_emb2 = (J_env + v_xc_total - v_xc_act + projector)

In [45]:
# check these are the SAME!
np.allclose(v_emb, v_emb2)

True

note $P$ is a projection operator that **enforces intersubsystem orbital orthogonality.**

# Next find optimized $\gamma^{A_{\text{act}}}_\text{embedded}$

- this is done by running SCF calc using embedded $h^{A_{\text{act}} \text{ in } B_{\text{env}}}$

Steps:

1. Define full molecular system
2. Change the number of electrons to be the same number as active system allows
3. run SCF to get optimized electron density!

TODO: Could maybe calculate $\gamma^{A_{\text{act}}}_\text{embedded}$ using high level method!

In [46]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_ao_ind)
# full_system_mol_EMBEDDED.nao = len(active_ao_ind)

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= 8_000
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional_or_HF

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core


EMBEDDED_full_system_scf.kernel()

-4.2264947650627445

In [47]:
EMBEDDED_full_system_scf.conv_check

True

# Get optimized $\gamma^{A_{\text{act}}}_\text{embedded}$

In [48]:


EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# optimized embedded denisty matrix
density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

## check number of electrons makes sense:
np.isclose(np.trace(density_emb@S), 2*len(active_ao_ind))

True

# Calculate Energy

The energy can be found according too:

$$E[\gamma^{A_{\text{act}}}_\text{embedded}; \; \; \gamma^{A_{\text{act}}},\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]
= \mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}] + E[\gamma^{B_{\text{env}}}] + g[\gamma^{A_{\text{act}}}, \gamma^{B_{\text{env}}}] + \mathcal{Trace}\big[(\gamma^{A_{\text{act}}}_\text{embedded}-\gamma^{A_{\text{act}}})(h^{A_{\text{act}} \text{ in } B_{\text{env}}} - h )\big]$$



- $\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]$
    - energy of isolated A computed with new optimized embedded density matrix
    - Note this allows for applicaton of different fucntionals (hence why odd E used)
- $E[\gamma^{B_{\text{env}}}]$ DFT calc of isolated subsystem B
- $g[\gamma^{A_{\text{act}}}, \gamma^{B_{\text{env}}}]$ non additive energy (boundary condition) between subsystems
    - this is obtained from global calc!
- Trace term provides a first order correction to the energy!

$\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]$

In [49]:
J_emb, K_emb =EMBEDDED_full_system_scf.get_jk(dm=density_emb) 

# NOTE how normal H_core is being used!
e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
e_act_emb

-31.857936912577458

In [50]:
# Test this with STANDARD full system obj
# (changing density matrix for embedded one)
J_test, K_test = full_system_scf.get_jk(dm=density_emb)

np.allclose(J_emb, J_test) and np.allclose(K_emb, K_test)

True

In [51]:
# should be able to calculate energy in the same fashion
# using STANDARD full system obj and new embedded density matrix

# THIS METHOD IS BETTER THAN ABOVE (matches energy of global calc now!!!!)

e_act_emb = full_system_scf.energy_elec(dm=density_emb)[0]
e_act_emb

-32.11498826785264

In [52]:
# note this is WRONG
EMBEDDED_full_system_scf.energy_elec(dm=density_emb)
# it is uses EMBEDDED h_core

(-13.354778493345682, 9.297650957177163)

Note for

$$\mathcal{Trace}\big[(\gamma^{A_{\text{act}}}_\text{embedded}-\gamma^{A_{\text{act}}})(h^{A_{\text{act}} \text{ in } B_{\text{env}}} - h )\big]$$


- as : $h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$
- we can re-write the first order correction as:

$$\mathcal{Trace}\big[(\gamma^{A_{\text{act}}}_\text{embedded}-\gamma^{A_{\text{act}}})(h^{A_{\text{act}} \text{ in } B_{\text{env}}} - h )\big] = \mathcal{Trace}\big[(\gamma^{A_{\text{act}}}_\text{embedded}-\gamma^{A_{\text{act}}})(v^{\text{embed}})\big]$$

In [53]:
correction = (matrix_dot(v_emb, density_emb - dm_active))
correction

-0.0003196770654942119

In [54]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + correction
e_mf_emb # <-- energy from embedded DFT calc

-74.73474040872496

In [55]:
full_system_scf.e_tot # <-- energy from normal DFT calc

-74.73474041595249

In [56]:
my_fci.e_tot # <-- energy from CI calc

-75.01535247852482

## higher level DFT calc

- we can calculate $\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]$ using a better functional

In [57]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= 8_000
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!
# full_system_scf_HIGH_LEVEL.kernel()

In [58]:
e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.energy_elec(dm=density_emb)
e_act_emb_HIGH_LVL # run energy calc using high level functional (note density matrix is embedded A)

(-32.174696433979605, 9.23794279108338)

In [59]:
e_act_emb_HIGH_LVL[0]

-32.174696433979605

In [60]:
E_high_lvl_DFT = e_act_emb_HIGH_LVL[0] + E_env + two_e_cross + full_system_scf.energy_nuc() + correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

-74.79444857485193

In [61]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))

print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

High level DFT in DFT error: 0.22090390367289103
LOW level DFT in DFT error: 0.280612069799858


# Perform high level WaveFunction calc

- we can calculate $\mathcal{E}[\gamma^{A_{\text{act}}}_\text{embedded}]$ using Wf methods now

# WF calculation

1. Perform HF calc using embedded Hcore to find gamma_act_emb

In [62]:
full_system_mol_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_HF.nelectron = 2*len(active_ao_ind)
# full_system_mol_HF.nao = len(active_ao_ind)

full_system_scf_HF = scf.RHF(full_system_mol_HF)
full_system_scf_HF.verbose=1
full_system_scf_HF.max_memory= 8_000
full_system_scf_HF.conv_tol = 1e-6


In [63]:
E_hf = full_system_scf_HF.energy_tot(dm=density_emb, 
                                         vhf=full_system_scf_HF.get_veff(dm=density_emb)
                                          )
E_hf

-22.729653184294527

In [207]:
total_ind = np.array([ind for ind in range(full_system_mol_HF.nao) if ind not in enviro_ao_ind])
total_ind

array([1, 2, 4, 5, 6])

In [388]:
from pyscf import ao2mo

C_emb = EMBEDDED_full_system_scf.mo_coeff
pyscf_mo_ints = ao2mo.kernel(full_system_mol_HF, C_emb) # uses embedded C matrix (full)

# Convert the 2e integrals (in Chemist’s notation)
n_bas_ft = full_system_mol_HF.nao
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, n_bas_ft)
pyscf_mo_ints.shape

(7, 7, 7, 7)

In [390]:
# eri_ao = full_system_mol_HF.intor('int2e')
# eri_ao = eri_ao[np.array(active_ao_ind),  :, :, :]
# eri_ao = eri_ao[:, np.array(active_ao_ind),  :, :]
# eri_ao = eri_ao[:, :,np.array(active_ao_ind), :]
# eri_ao = eri_ao[:, :,:, np.array(active_ao_ind)]

# C_emb = EMBEDDED_full_system_scf.mo_coeff[active_ao_ind, :]
# tmp = np.einsum('pi,pqrs->iqrs', C_emb, eri_ao, optimize=True)
# tmp = np.einsum('qa,iqrs->iars', C_emb, tmp, optimize=True)
# tmp = np.einsum('iars,rj->iajs', tmp, C_emb, optimize=True)
# pyscf_mo_ints2 = np.einsum('iajs,sb->iajb', tmp, C_emb, optimize=True)
# del tmp
# pyscf_mo_ints2.shape

# H_core = scf.hf.get_hcore(full_system_mol_HF)[np.array(active_ao_ind), :]
# H_core = H_core[:, np.array(active_ao_ind)]

# hcore_mo =  C_emb.conj().T @ H_core@ C_emb # <psi H psi> # NOT FOCK

In [65]:
# eri_ao = full_system_mol_HF.intor('int2e')

# tmp = np.einsum('pi,pqrs->iqrs', C_emb, eri_ao, optimize=True)
# tmp = np.einsum('qa,iqrs->iars', C_emb, tmp, optimize=True)
# tmp = np.einsum('iars,rj->iajs', tmp, C_emb, optimize=True)
# pyscf_mo_ints2 = np.einsum('iajs,sb->iajb', tmp, C_emb, optimize=True)
# del tmp
# pyscf_mo_ints2.shape

In [391]:
hcore_mo =  C_emb.conj().T @ scf.hf.get_hcore(full_system_mol_HF) @ C_emb # <psi H psi> # NOT FOCK

In [392]:
# EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# # optimized embedded denisty matrix
# density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

# ## check number of electrons makes sense:
# np.isclose(np.trace(density_emb@S), 2*len(active_ao_ind))

dm22 = 2 * C_emb @ C_emb.conj().T
print(np.trace(dm22@S))
full_system_mol.nelectron

14.0


10

In [385]:
# CHECK match with HF energy!

n_docc = full_system_mol_HF.nelectron//2

E = 2*np.einsum('ii->', hcore_mo[:n_docc, :n_docc], optimize=True)

two_J = 2*np.einsum('iijj->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
K = np.einsum('ijji->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
E+= (two_J-K)

E+= full_system_mol_HF.energy_nuc()

print('Energies match:', np.isclose(E_hf, E))
E

Energies match: False


2.6683155690834273

In [386]:
n_qubits = 2*hcore_mo.shape[0]

one_body_terms = np.zeros((n_qubits, n_qubits))
two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        one_body_terms[2*p, 2*q] = hcore_mo[p,q] # spin UP
        one_body_terms[(2*p + 1), (2*q +1)] = hcore_mo[p,q] # spin DOWN
        
        # continue 2-body terms
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                                
                ### SAME spin                
                two_body_terms[2*p, 2*q , 2*r, 2*s] = pyscf_mo_ints[p,q,r,s] # up up up up
                two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = pyscf_mo_ints[p,q,r,s] # down down down down
                
                ### mixed spin                
                two_body_terms[2*p, 2*q , (2*r + 1), (2*s +1)] = pyscf_mo_ints[p,q,r,s] # up up down down
                two_body_terms[(2*p+1), (2*q +1) , 2*r, 2*s] = pyscf_mo_ints[p,q,r,s] # down down up up 
               
                
                
                
### remove vanishing terms
EQ_Tolerance=1e-8
one_body_terms[np.abs(one_body_terms)<EQ_Tolerance]=0
two_body_terms[np.abs(two_body_terms)<EQ_Tolerance]=0

IndexError: index 3 is out of bounds for axis 3 with size 3

In [285]:
N = full_system_mol_HF.nelectron

E_spin_MOs = np.einsum('ii->', one_body_terms[:N, :N], optimize=True)

J = np.einsum('iijj->', two_body_terms[:N, :N, :N, :N], optimize=True)
K = np.einsum('ijji->', two_body_terms[:N, :N, :N, :N], optimize=True)
E_spin_MOs+= 0.5*(J-K)

E_spin_MOs+= full_system_mol.energy_nuc()

print('Energies match:', np.isclose(E_hf, E_spin_MOs))
E_spin_MOs

Energies match: False


-20.49558018010745

In [286]:
from openfermion.ops import FermionOperator

In [287]:
H_fermionic = FermionOperator((),  full_system_mol.energy_nuc())

# two_body_terms = two_body_terms.transpose(0,2,3,1) # for physist notation!

# one body terms
for p in range(one_body_terms.shape[0]):
    for q in range(one_body_terms.shape[0]):
        
        H_fermionic += one_body_terms[p,q] * FermionOperator(((p, 1), (q, 0)))
        
        # two body terms
        for r in range(two_body_terms.shape[0]):
            for s in range(two_body_terms.shape[0]):
                
                ######## physist notation
                ## (requires:
                ##           two_body_terms transpose (0,2,3,1) before loop starts!
                ##)
#                 H_qubit += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))
                
                ######## chemist notation
                H_fermionic += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))
H_fermionic

9.128283728282938 [] +
-6.373597876504241 [0^ 0] +
0.31721150914278695 [0^ 0^ 0 0] +
0.04595130195531671 [0^ 0^ 0 4] +
0.014543180678703171 [0^ 0^ 2 2] +
0.04595130195531668 [0^ 0^ 4 0] +
0.07680173985536486 [0^ 0^ 4 4] +
0.31721150914278695 [0^ 1^ 1 0] +
0.04595130195531671 [0^ 1^ 1 4] +
0.014543180678703171 [0^ 1^ 3 2] +
0.04595130195531668 [0^ 1^ 5 0] +
0.07680173985536486 [0^ 1^ 5 4] +
0.014543180678703171 [0^ 2^ 0 2] +
0.3155224021505817 [0^ 2^ 2 0] +
0.09489361121888093 [0^ 2^ 2 4] +
0.011809714386954294 [0^ 2^ 4 2] +
0.014543180678703171 [0^ 3^ 1 2] +
0.3155224021505817 [0^ 3^ 3 0] +
0.09489361121888093 [0^ 3^ 3 4] +
0.011809714386954294 [0^ 3^ 5 2] +
-1.698075507355392 [0^ 4] +
0.04595130195531668 [0^ 4^ 0 0] +
0.07680173985536486 [0^ 4^ 0 4] +
0.011809714386954294 [0^ 4^ 2 2] +
0.30471221564034656 [0^ 4^ 4 0] +
0.044389673074901624 [0^ 4^ 4 4] +
0.04595130195531668 [0^ 5^ 1 0] +
0.07680173985536486 [0^ 5^ 1 4] +
0.011809714386954294 [0^ 5^ 3 2] +
0.30471221564034656 [0^ 5^ 5 0

In [288]:
from openfermion.transforms import jordan_wigner

H_qubit_JW = jordan_wigner(H_fermionic)
H_qubit_JW


(-7.973230997634613+0j) [] +
(-0.007271590339351585+0j) [X0 X1 Y2 Y3] +
(-0.03840086992768243+0j) [X0 X1 Y4 Y5] +
(0.007271590339351585+0j) [X0 Y1 Y2 X3] +
(0.03840086992768243+0j) [X0 Y1 Y4 X5] +
(0.005904857193477149+0j) [X0 Z1 X2 X3 Z4 X5] +
(0.005904857193477149+0j) [X0 Z1 X2 Y3 Z4 Y5] +
(-0.7148785121371835+0j) [X0 Z1 Z2 Z3 X4] +
(-0.022194836537450812+0j) [X0 Z1 Z2 Z3 X4 Z5] +
(-0.04744680560944044+0j) [X0 Z1 Z2 X4] +
(-0.041541948415963294+0j) [X0 Z1 Z3 X4] +
(-0.022975650977658347+0j) [X0 Z2 Z3 X4] +
(0.007271590339351585+0j) [Y0 X1 X2 Y3] +
(0.03840086992768243+0j) [Y0 X1 X4 Y5] +
(-0.007271590339351585+0j) [Y0 Y1 X2 X3] +
(-0.03840086992768243+0j) [Y0 Y1 X4 X5] +
(0.005904857193477149+0j) [Y0 Z1 Y2 X3 Z4 X5] +
(0.005904857193477149+0j) [Y0 Z1 Y2 Y3 Z4 Y5] +
(-0.7148785121371835+0j) [Y0 Z1 Z2 Z3 Y4] +
(-0.022194836537450812+0j) [Y0 Z1 Z2 Z3 Y4 Z5] +
(-0.04744680560944044+0j) [Y0 Z1 Z2 Y4] +
(-0.041541948415963294+0j) [Y0 Z1 Z3 Y4] +
(-0.022975650977658347+0j) [Y0 Z2 Z3 Y4] +
(

In [289]:
from openfermion.linalg import get_sparse_operator

H_JW_mat = get_sparse_operator(H_qubit_JW)

In [302]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_JW_mat, which='SA', k=1)

In [291]:
eigvals_EMBED

array([-20.49558018])

In [292]:
np.where(eigvecs_EMBED>1e-2)[0]

array([], dtype=int64)

In [293]:
np.binary_repr(np.where(eigvecs_EMBED>1e-2)[0][0]).count('1') # look at number of electrons! there are 10 == BAD

IndexError: index 0 is out of bounds for axis 0 with size 0

In [294]:
full_system_mol.nelectron

10

In [295]:
# in embedded density matrix we have this many electrons
np.trace(density_emb@S)

6.000000000000001

In [299]:

E_high_lvl_WF = eigvals_EMBED[0] + E_env + two_e_cross  + correction
E_high_lvl_DFT # <-- energy from embedded WF calc

-133.75894438639298

In [241]:
from functools import reduce

zero_state = np.array([[1],[0]])
one_state = np.array([[0],[1]])

HF_state = reduce(np.kron, [*[one_state for _ in range(N)], *[zero_state for _ in range(N, 2*full_system_mol_HF.nao)]]
                 ) # | 1 1 0 0 >
HF_state


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [242]:
HF_state.conj().T @ H_JW_mat @ HF_state

array([[-22.72965318+0.j]])

In [244]:
def get_active_space_integrals(one_body_integrals,
                               two_body_integrals,
                               occupied_indices=None,
                               active_indices=None):
    """Restricts a molecule at a spatial orbital level to an active space
    This active space may be defined by a list of active indices and
        doubly occupied indices. Note that one_body_integrals and
        two_body_integrals must be defined
        n an orthonormal basis set.
    Args:
        one_body_integrals: One-body integrals of the target Hamiltonian
        two_body_integrals: Two-body integrals of the target Hamiltonian
        occupied_indices: A list of spatial orbital indices
            indicating which orbitals should be considered doubly occupied.
        active_indices: A list of spatial orbital indices indicating
            which orbitals should be considered active.
    Returns:
        tuple: Tuple with the following entries:
        **core_constant**: Adjustment to constant shift in Hamiltonian
        from integrating out core orbitals
        **one_body_integrals_new**: one-electron integrals over active
        space.
        **two_body_integrals_new**: two-electron integrals over active
        space.
    """
    # Fix data type for a few edge cases
    occupied_indices = [] if occupied_indices is None else occupied_indices
    if (len(active_indices) < 1):
        raise ValueError('Some active indices required for reduction.')

    # Determine core constant
    core_constant = 0.0
    for i in occupied_indices:
        core_constant += 2 * one_body_integrals[i, i]
        for j in occupied_indices:
            core_constant += (2 * two_body_integrals[i, j, j, i] -
                              two_body_integrals[i, j, i, j])

    # Modified one electron integrals
    one_body_integrals_new = np.copy(one_body_integrals)
    for u in active_indices:
        for v in active_indices:
            for i in occupied_indices:
                one_body_integrals_new[u, v] += (
                    2 * two_body_integrals[i, u, v, i] -
                    two_body_integrals[i, u, i, v])

    # Restrict integral ranges and change M appropriately
    return (core_constant,
            one_body_integrals_new[np.ix_(active_indices, active_indices)],
            two_body_integrals[np.ix_(active_indices, active_indices,
                                         active_indices, active_indices)])

In [154]:
active = [(2*act_ind, 2*act_ind+1)  for act_ind in active_ao_ind]
active = [term for tup in active for term in tup]
active

[2, 3, 4, 5, 8, 9]

In [155]:
enviro = [(2*env_ind, 2*env_ind+1)  for env_ind in enviro_ao_ind]
enviro = [term for tup in enviro for term in tup]
enviro

[0, 1, 6, 7]

In [156]:
used_inds = [*active_ao_ind, *enviro_ao_ind]
core_inds = [i for i in range(full_system_mol.aoslice_by_atom()[-1,-1]) if i not in used_inds
           ]
core = [(2*core_ind, 2*core_ind+1)  for core_ind in core_inds]
core = [term for tup in core for term in tup]
core

[10, 11, 12, 13]

In [263]:
2*np.array(enviro_ao_ind)+1

array([1, 7])

In [259]:
total_ind = np.array([ind for ind in range(full_system_mol_HF.nao) if ind not in enviro_ao_ind])

active_space_ind = sorted([*(2*total_ind).tolist(), *(2*total_ind+1).tolist()])
active_space_ind

[2, 3, 4, 5, 8, 9, 10, 11, 12, 13]

In [270]:
core_constant_new, one_body_integrals_new, two_body_integrals_new = get_active_space_integrals(
                               one_body_terms,
                               two_body_terms,
                               occupied_indices=None,
                               active_indices= active_space_ind) #[*active, *core]

In [271]:
H_fermionic = FermionOperator((),  core_constant_new)

# one body terms
for p in range(one_body_integrals_new.shape[0]):
    for q in range(one_body_integrals_new.shape[0]):
        
        H_fermionic += one_body_integrals_new[p,q] * FermionOperator(((p, 1), (q, 0)))
        
        # two body terms
        for r in range(two_body_integrals_new.shape[0]):
            for s in range(two_body_integrals_new.shape[0]):
                
                ######## physist notation
                ## (requires:
                ##           two_body_terms = two_body_terms.transpose(0,2,3,1) before loop starts!
                ##)
#                 H_qubit += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))
                
                ######## chemist notation
                H_fermionic += 0.5*two_body_integrals_new[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))
H_fermionic

-6.373597876504241 [0^ 0] +
0.3172115091427869 [0^ 0^ 0 0] +
0.0459513019553167 [0^ 0^ 0 4] +
0.014543180678703171 [0^ 0^ 2 2] +
0.04595130195531669 [0^ 0^ 4 0] +
0.07680173985536488 [0^ 0^ 4 4] +
0.03783117211147126 [0^ 0^ 6 6] +
0.007014546015985034 [0^ 0^ 6 8] +
0.007014546015985036 [0^ 0^ 8 6] +
0.005436126053324211 [0^ 0^ 8 8] +
0.3172115091427869 [0^ 1^ 1 0] +
0.0459513019553167 [0^ 1^ 1 4] +
0.014543180678703171 [0^ 1^ 3 2] +
0.04595130195531669 [0^ 1^ 5 0] +
0.07680173985536488 [0^ 1^ 5 4] +
0.03783117211147126 [0^ 1^ 7 6] +
0.007014546015985034 [0^ 1^ 7 8] +
0.007014546015985036 [0^ 1^ 9 6] +
0.005436126053324211 [0^ 1^ 9 8] +
0.014543180678703171 [0^ 2^ 0 2] +
0.3155224021505817 [0^ 2^ 2 0] +
0.09489361121888092 [0^ 2^ 2 4] +
0.011809714386954294 [0^ 2^ 4 2] +
0.014543180678703171 [0^ 3^ 1 2] +
0.3155224021505817 [0^ 3^ 3 0] +
0.09489361121888092 [0^ 3^ 3 4] +
0.011809714386954294 [0^ 3^ 5 2] +
-1.698075507355392 [0^ 4] +
0.04595130195531669 [0^ 4^ 0 0] +
0.07680173985536488 

In [272]:
H_qubit_JW = jordan_wigner(H_fermionic)
H_qubit_JW

(-50.364200843635174+0j) [] +
(-0.007271590339351585+0j) [X0 X1 Y2 Y3] +
(-0.03840086992768244+0j) [X0 X1 Y4 Y5] +
(-0.018915586055735626+0j) [X0 X1 Y6 Y7] +
(-0.0035072730079925177+0j) [X0 X1 Y6 Z7 Z8 Y9] +
(-0.0035072730079925177+0j) [X0 X1 X7 X8] +
(-0.002718063026662106+0j) [X0 X1 Y8 Y9] +
(0.007271590339351585+0j) [X0 Y1 Y2 X3] +
(0.03840086992768244+0j) [X0 Y1 Y4 X5] +
(0.018915586055735626+0j) [X0 Y1 Y6 X7] +
(0.0035072730079925177+0j) [X0 Y1 Y6 Z7 Z8 X9] +
(-0.0035072730079925177+0j) [X0 Y1 Y7 X8] +
(0.002718063026662106+0j) [X0 Y1 Y8 X9] +
(0.005904857193477149+0j) [X0 Z1 X2 X3 Z4 X5] +
(0.005904857193477149+0j) [X0 Z1 X2 Y3 Z4 Y5] +
(-0.4438900641369011+0j) [X0 Z1 Z2 Z3 X4] +
(-0.022194836537450816+0j) [X0 Z1 Z2 Z3 X4 Z5] +
(-0.005198180889701611+0j) [X0 Z1 Z2 Z3 X4 X6 Z7 X8] +
(-0.0062925742523655785+0j) [X0 Z1 Z2 Z3 X4 Y6 Z7 Y8] +
(-0.03721546053493454+0j) [X0 Z1 Z2 Z3 X4 Z6] +
(-0.0012787362860944783+0j) [X0 Z1 Z2 Z3 X4 X7 Z8 X9] +
(-0.0012787362860944783+0j) [X0 Z1 Z2 Z3 

In [273]:
len(list(H_qubit_JW))

252

In [274]:
H_JW_mat = get_sparse_operator(H_qubit_JW)
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_JW_mat, which='SA', k=1)
eigvals_EMBED[0]

-82.0109085172377

In [275]:
E_high_lvl_DFT = eigvals_EMBED[0] + E_env + two_e_cross  + correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

-133.75894438639298

# define ansatz from HF ground state using coupled cluster!

limiting to correct number of electrons

In [ ]:
def Get_ia_terms(n_electrons, n_orbitals, single_cc_amplitudes=None,  singles_hamiltonian=None,
                               tol_filter_small_terms = None):
    """

    Get ia excitation terms as fermionic creation and annihilation operators for UCCSD.
    ia terms are standard single excitation terms (aka only occupied -> unoccupied transitions allowed)
    (faster and only marginally less accurate.)
    #TODO can add method to get pqrs terms
    #TODO these are all non-degenerate excitations which can possibly non-zero, including nocc->nocc, occ->occ, and spin-flips.
    #TODO EXPENSIVE, but will likely  get a slightly better answer.

    Args:
        n_electrons (int): number of electrons
        n_orbitals (int): number of orbitals
        singles_hamiltonian (numpy.ndarray, optional): h_pq (n_qubits x n_qubits) matrix.
        tol_filter_small_terms (bool, optional):  Whether to filter small terms in Hamiltonian (threshold currently hardcoded)
        single_cc_amplitudes (numpy.ndarray, optional): A 2-dimension array t[a,i] for CCSD single excitation amplitudes
                                where a is virtual index and i is occupied index

    returns:
        Sec_Quant_CC_ia_ops (list): list of FermionOperators (openfermion.ops._fermion_operator.FermionOperator)
        theta_parameters (list): list of theta values (parameterisation of excitation amplitudes)

     e.g.:

     n_electrons=2
     n_orbitals=4
     Sec_Quant_CC_ops, theta_parameters = Get_ia_terms(n_electrons, n_orbitals)

     Sec_Quant_CC_ops=  [
                         -1.0[0 ^ 2] + 1.0[2 ^ 0],            # -(a†0 a2) + (a†2 a0)
                         -1.0[1 ^ 3] + 1.0[3 ^ 1],            # -(a†1 a3) + (a†3 a1)
                        ]
    theta_parameters = [0,0,0]

    """

    Sec_Quant_CC_ia_ops = []  # second quantised single e- CC operators
    theta_parameters_ia = []

    # single_amplitudes and double_amplitudes from Get_CCSD_Amplitudes Hamiltonian function!
    orbitals_index = range(0, n_orbitals)

    alph_occs = [k for k in orbitals_index if k % 2 == 0 and k < n_electrons]  # spin up occupied
    beta_occs = [k for k in orbitals_index if k % 2 == 1 and k < n_electrons]  # spin down UN-occupied
    alph_noccs = [k for k in orbitals_index if k % 2 == 0 and k >= n_electrons]  # spin down occupied
    beta_noccs = [k for k in orbitals_index if k % 2 == 1 and k >= n_electrons]  # spin up UN-occupied


    # SINGLE electron excitation: spin UP transition
    for i in alph_occs:
        for a in alph_noccs:
            if tol_filter_small_terms:
                if abs(singles_hamiltonian[i][a]) > tol_filter_small_terms or abs(
                        singles_hamiltonian[a][i]) > tol_filter_small_terms:
                    one_elec = FermionOperator(((a, 1), (i, 0))) - FermionOperator(((i, 1), (a, 0)))
                    if single_cc_amplitudes is not None:
                        theta_parameters_ia.append(single_cc_amplitudes[a][i])
                    else:
                        theta_parameters_ia.append(0)

                    Sec_Quant_CC_ia_ops.append(one_elec)
            else:
                # NO filtering
                one_elec = FermionOperator(((a, 1), (i, 0))) - FermionOperator(((i, 1), (a, 0)))
                if single_cc_amplitudes is not None:
                    theta_parameters_ia.append(single_cc_amplitudes[a][i])
                else:
                    theta_parameters_ia.append(0)

                Sec_Quant_CC_ia_ops.append(one_elec)

    # SINGLE electron excitation: spin DOWN transition
    for i in beta_occs:
        for a in beta_noccs:
            if tol_filter_small_terms:
                # uses Hamiltonian to ignore small terms!
                if abs(singles_hamiltonian[i][a]) > tol_filter_small_terms or abs(
                        singles_hamiltonian[a][i]) > tol_filter_small_terms:
                    one_elec = FermionOperator(((a, 1), (i, 0))) - FermionOperator(((i, 1), (a, 0)))
                    if single_cc_amplitudes is not None:
                        theta_parameters_ia.append(single_cc_amplitudes[a][i])
                    else:
                        theta_parameters_ia.append(0)

                    Sec_Quant_CC_ia_ops.append(one_elec)
            else:
                # NO filtering
                one_elec = FermionOperator(((a, 1), (i, 0))) - FermionOperator(((i, 1), (a, 0)))
                if single_cc_amplitudes is not None:
                    theta_parameters_ia.append(single_cc_amplitudes[a][i])
                else:
                    theta_parameters_ia.append(0)

                Sec_Quant_CC_ia_ops.append(one_elec)

    return Sec_Quant_CC_ia_ops, theta_parameters_ia


def Get_ijab_terms(n_electrons, n_orbitals, double_cc_amplitudes=None, doubles_hamiltonian=None,
                 tol_filter_small_terms=None):
    """

    Get ijab excitation terms as fermionic creation and annihilation operators for UCCSD.
    ijab terms are standard double excitation terms (aka only occupied -> unoccupied transitions allowed)
    (faster and only marginally less accurate.)
    #TODO can add method to get pqrs terms
    #TODO these are all non-degenerate excitations which can possibly non-zero, including nocc->nocc, occ->occ, and spin-flips.
    #TODO EXPENSIVE, but will likely  get a slightly better answer.

    Args:
        n_electrons (int): number of electrons
        n_orbitals (int): number of orbitals
        doubles_hamiltonian (numpy.ndarray, optional): h_pqrs (n_qubits x n_qubits x n_qubits x n_qubits) matrix
        tol_filter_small_terms (bool, optional):  Whether to filter small terms in Hamiltonian (threshold currently hardcoded)
        double_cc_amplitudes (numpy.ndarray, optional): A 4-dimension array t[a,i,b,j] for CCSD double excitation amplitudes
                                                        where a, b are virtual indices and i, j are occupied indices.

    returns:
        Sec_Quant_CC_ijab_ops (list): list of FermionOperators (openfermion.ops._fermion_operator.FermionOperator)
        theta_parameters (list): list of theta values (parameterisation of excitation amplitudes)

     e.g.:

     n_electrons=2
     n_orbitals=4
     Sec_Quant_CC_ops, theta_parameters = Get_ijab_terms(n_electrons, n_orbitals)

     Sec_Quant_CC_ops=  [
                            -1.0[0 ^ 1 ^ 2 3] + 1.0 [3^ 2^ 1 0]  # -(a†0 a†1 a2 a3) + a†3 a†2 a1 a0)
                        ]
    theta_parameters = [0]
    """


    # single_amplitudes and double_amplitudes from Get_CCSD_Amplitudes Hamiltonian function!
    orbitals_index = range(0, n_orbitals)

    alph_occs = [k for k in orbitals_index if k % 2 == 0 and k < n_electrons]  # spin up occupied
    beta_occs = [k for k in orbitals_index if k % 2 == 1 and k < n_electrons]  # spin down UN-occupied
    alph_noccs = [k for k in orbitals_index if k % 2 == 0 and k >= n_electrons]  # spin down occupied
    beta_noccs = [k for k in orbitals_index if k % 2 == 1 and k >= n_electrons]  # spin up UN-occupied

    Sec_Quant_CC_ijab_ops = []  # second quantised two e- CC operators
    theta_parameters_ijab = []

    # DOUBLE excitation: UP + UP
    for i in alph_occs:
        for j in [k for k in alph_occs if k > i]:
            for a in alph_noccs:
                for b in [k for k in alph_noccs if k > a]:

                    if tol_filter_small_terms:
                        # uses Hamiltonian to ignore small terms!
                        if abs(doubles_hamiltonian[j][i][a][b]) > tol_filter_small_terms or abs(
                                doubles_hamiltonian[b][a][i][j]) > tol_filter_small_terms:
                            two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                       FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))
                            if double_cc_amplitudes is not None:
                                theta_parameters_ijab.append(double_cc_amplitudes[a][i][b][j])
                            else:
                                theta_parameters_ijab.append(0)
                        Sec_Quant_CC_ijab_ops.append(two_elec)
                    else:
                        # NO filtering
                        two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                   FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))

                        if double_cc_amplitudes is not None:
                            theta_parameters_ijab.append(double_cc_amplitudes[b][a][j][i])
                        else:
                            theta_parameters_ijab.append(0)

                        Sec_Quant_CC_ijab_ops.append(two_elec)

    # DOUBLE excitation: DOWN + DOWN
    for i in beta_occs:
        for j in [k for k in beta_occs if k > i]:
            for a in beta_noccs:
                for b in [k for k in beta_noccs if k > a]:

                    if tol_filter_small_terms:
                        # uses Hamiltonian to ignore small terms!
                        if abs(doubles_hamiltonian[j][i][a][b]) > tol_filter_small_terms or abs(
                                doubles_hamiltonian[b][a][i][j]) > tol_filter_small_terms:
                            two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                       FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))
                            if double_cc_amplitudes is not None:
                                theta_parameters_ijab.append(double_cc_amplitudes[a][i][b][j])
                            else:
                                theta_parameters_ijab.append(0)
                        Sec_Quant_CC_ijab_ops.append(two_elec)
                    else:
                        # NO filtering
                        two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                   FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))

                        if double_cc_amplitudes is not None:
                            theta_parameters_ijab.append(double_cc_amplitudes[a][i][b][j])
                        else:
                            theta_parameters_ijab.append(0)

                        Sec_Quant_CC_ijab_ops.append(two_elec)

    # DOUBLE excitation: up + DOWN
    for i in alph_occs:
        for j in [k for k in beta_occs if k > i]:
            for a in alph_noccs:
                for b in [k for k in beta_noccs if k > a]:

                    if tol_filter_small_terms:
                        # uses Hamiltonian to ignore small terms!
                        if abs(doubles_hamiltonian[j][i][a][b]) > tol_filter_small_terms or abs(
                                doubles_hamiltonian[b][a][i][j]) > tol_filter_small_terms:
                            two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                       FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))
                            if double_cc_amplitudes is not None:
                                theta_parameters_ijab.append(double_cc_amplitudes[a][i][b][j])
                            else:
                                theta_parameters_ijab.append(0)
                        Sec_Quant_CC_ijab_ops.append(two_elec)
                    else:
                        # NO filtering
                        two_elec = FermionOperator(((b, 1), (a, 1), (j, 0), (i, 0))) - \
                                   FermionOperator(((i, 1), (j, 1), (a, 0), (b, 0)))

                        if double_cc_amplitudes is not None:
                            theta_parameters_ijab.append(double_cc_amplitudes[a][i][b][j])
                        else:
                            theta_parameters_ijab.append(0)

                        Sec_Quant_CC_ijab_ops.append(two_elec)

    return Sec_Quant_CC_ijab_ops, theta_parameters_ijab


def Fermi_ops_to_qubit_ops(List_Fermi_Ops, transformation='JW'):

        """
        Takes list of fermionic excitation operators:

                    [
                       -(a†0 a2) + (a†2 a0),
                       -(a†1 a3) + (a†3 a1),
                       -(a†0 a†1 a2 a3) + a†3 a†2 a1 a0)
                    ]
        and returns JW/BK transform of each term and appends it to a list yielding a list of QubitOperators

        [
            -0.5j [X0 Z1 Y2] + 0.5j [Y0 Z1 X2],
            -0.5j [X1 Z2 Y3] + 0.5j [Y1 Z2 X3],
            0.125j [X0 X1 X2 Y3] + 0.125j [X0 X1 Y2 X3] + -0.125j [X0 Y1 X2 X3] + 0.125j [X0 Y1 Y2 Y3] +
            -0.125j [Y0 X1 X2 X3] + 0.125j [Y0 X1 Y2 Y3] + -0.125j [Y0 Y1 X2 Y3] + -0.125j [Y0 Y1 Y2 X3]
        ]

        returns:
            List_Qubit_Ops (list): List of QubitOperators (openfermion.ops._qubit_operator.QubitOperator)
                                   under JW/BK transform.

        """

        List_Qubit_Ops = []

        if transformation == 'JW':
            for OP in List_Fermi_Ops:
                JW_OP = jordan_wigner(OP)
                List_Qubit_Ops.append(JW_OP)

        elif transformation == 'BK':
            for OP in List_Fermi_Ops:
                BK_OP = bravyi_kitaev(OP)
                List_Qubit_Ops.append(BK_OP)

        else:
            raise ValueError('unknown transformation: {}'.format(transformation))

        return List_Qubit_Ops


In [ ]:
N_electrons = 2*len(active_ao_ind)
N_orbs = 2*full_system_mol_HF.nao

ia_ferm, theta_ia = Get_ia_terms(N_electrons, N_orbs)

ijab_ferm, theta_ijab = Get_ijab_terms(N_electrons, N_orbs)

In [ ]:
ia_qubit_list = Fermi_ops_to_qubit_ops(ia_ferm, transformation='JW')
ijab_qubit_list = Fermi_ops_to_qubit_ops(ijab_ferm, transformation='JW')
ia_qubit_list

In [ ]:
from openfermion import qubit_operator_sparse
from scipy.sparse.linalg import expm

def Generate_state(theta_list, HF_state, ia_qubit_ops, ijab_qubit_ops, N_qubits):
    
    running_state = HF_state
    
    ind=0
    for qubit_op_ia in ia_qubit_ops:
        print(ind)
        pauli_theta = qubit_operator_sparse(qubit_op_ia, n_qubits=N_qubits) * theta_list[ind]/2
        running_state = expm(-1j * pauli_theta) @ running_state
        ind+=1
        
#     for qubit_op_ijab in ijab_qubit_ops:
#         pauli_theta = qubit_operator_sparse(qubit_op_ijab, n_qubits=N_qubits) * theta_list[ind]/2
#         running_state = expm(-1j * pauli_theta) @ running_state
#         ind+=1
    
    return running_state
    

In [ ]:
state = Generate_state([*theta_ia, *theta_ijab],
                       HF_state, 
                       ia_qubit_list,
                       ijab_qubit_list,
                       2*full_system_mol_HF.nao)

In [ ]:
e_hf_check = EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb_HF, 
                                         vhf=EMBEDDED_full_system_scf_HF.get_veff())

In [ ]:
EMBEDDED_full_system_scf_HF.conv_check

In [ ]:
EMBEDDED_occ_orbs_HF = EMBEDDED_full_system_scf_HF.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0]

# optimized embedded denisty matrix
density_emb_HF = 2 * EMBEDDED_occ_orbs_HF @ EMBEDDED_occ_orbs_HF.conj().T

## check number of electrons makes sense:
np.isclose(np.trace(density_emb_HF@S), 2*len(active_ao_ind))

In [ ]:
e_hf_check = EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb_HF, 
                                         vhf=EMBEDDED_full_system_scf_HF.get_veff())
e_hf_check

In [ ]:
from pyscf import ao2mo

pyscf_mo_ints = ao2mo.kernel(full_system_mol_EMBEDDED, EMBEDDED_full_system_scf_HF.mo_coeff)#, aosym=1)

# Convert the 2e integrals (in Chemist’s notation)
n_bas_ft = full_system_mol_EMBEDDED.nao
pyscf_mo_ints = ao2mo.restore(1, pyscf_mo_ints, n_bas_ft)

In [ ]:
hcore_EMBED_mo =  EMBEDDED_full_system_scf_HF.mo_coeff.conj().T @ EMBEDDED_full_system_scf_HF.get_hcore() @ EMBEDDED_full_system_scf_HF.mo_coeff # <psi H psi> # NOT FOCK

In [ ]:
n_docc = full_system_mol_EMBEDDED_HF.nelectron//2

E = 2*np.einsum('ii->', hcore_EMBED_mo[:n_docc, :n_docc], optimize=True)

two_J = 2*np.einsum('iijj->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
K = np.einsum('ijji->', pyscf_mo_ints[:n_docc, :n_docc, :n_docc, :n_docc], optimize=True)
E+= (two_J-K)

E+= full_system_mol_EMBEDDED_HF.energy_nuc()
E

# Next

2. Find embedded wavefunction!

In [ ]:
n_qubits = 2*hcore_EMBED_mo.shape[0]

one_body_terms = np.zeros((n_qubits, n_qubits))
two_body_terms = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        one_body_terms[2*p, 2*q] = hcore_EMBED_mo[p,q] # spin UP
        one_body_terms[(2*p + 1), (2*q +1)] = hcore_EMBED_mo[p,q] # spin DOWN
        
        # continue 2-body terms
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                                
                ### SAME spin                
                two_body_terms[2*p, 2*q , 2*r, 2*s] = pyscf_mo_ints[p,q,r,s] # up up up up
                two_body_terms[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = pyscf_mo_ints[p,q,r,s] # down down down down
                
                ### mixed spin                
                two_body_terms[2*p, 2*q , (2*r + 1), (2*s +1)] = pyscf_mo_ints[p,q,r,s] # up up down down
                two_body_terms[(2*p+1), (2*q +1) , 2*r, 2*s] = pyscf_mo_ints[p,q,r,s] # down down up up 
               
                
                
                
### remove vanishing terms
EQ_Tolerance=1e-8
one_body_terms[np.abs(one_body_terms)<EQ_Tolerance]=0
two_body_terms[np.abs(two_body_terms)<EQ_Tolerance]=0

In [ ]:
N = full_system_mol_EMBEDDED_HF.nelectron

E_spin_MOs = np.einsum('ii->', one_body_terms[:N, :N], optimize=True)

J = np.einsum('iijj->', two_body_terms[:N, :N, :N, :N], optimize=True)
K = np.einsum('ijji->', two_body_terms[:N, :N, :N, :N], optimize=True)
E_spin_MOs+= 0.5*(J-K)

E_spin_MOs+= full_system_mol.energy_nuc()

E_spin_MOs

In [ ]:
from openfermion.ops import FermionOperator

In [ ]:
H_fermionic = FermionOperator((),  full_system_mol.energy_nuc())

# one body terms
for p in range(one_body_terms.shape[0]):
    for q in range(one_body_terms.shape[0]):
        
        H_fermionic += one_body_terms[p,q] * FermionOperator(((p, 1), (q, 0)))
        
        # two body terms
        for r in range(two_body_terms.shape[0]):
            for s in range(two_body_terms.shape[0]):
                
                ######## physist notation
                ## (requires:
                ##           two_body_terms = two_body_terms.transpose(0,2,3,1) before loop starts!
                ##)
#                 H_qubit += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))
                
                ######## chemist notation
                H_fermionic += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))
H_fermionic

In [ ]:
from openfermion.transforms import jordan_wigner

H_qubit_JW = jordan_wigner(H_fermionic)
H_qubit_JW

In [ ]:
len(list(H_qubit_JW))

In [ ]:
from openfermion.linalg import get_sparse_operator

H_JW_mat = get_sparse_operator(H_qubit_JW)

In [ ]:
H_JW_mat.shape

In [ ]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_JW_mat, which='SA', k=1)

# Calculate Energy with Embedded wavefunction

In [ ]:
full_system_mol_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_HF.nelectron = 2*len(active_ao_ind)

full_system_scf_HF = scf.RHF(full_system_mol_HF)
full_system_scf_HF.verbose=1
full_system_scf_HF.max_memory= 8_000
full_system_scf_HF.conv_tol = 1e-6


In [ ]:
pyscf_mo_ints = ao2mo.kernel(full_system_mol_HF, EMBEDDED_full_system_scf_HF.mo_coeff)#, aosym=1)

# Convert the 2e integrals (in Chemist’s notation)
n_bas_ft = full_system_mol_HF.nao
pyscf_mo_ints_std = ao2mo.restore(1, pyscf_mo_ints, n_bas_ft)

In [ ]:
hcore_standard_mo =  EMBEDDED_full_system_scf_HF.mo_coeff.conj().T @ scf.hf.get_hcore(full_system_mol_HF) @ EMBEDDED_full_system_scf_HF.mo_coeff # <psi H psi> # NOT FOCK


In [ ]:
n_qubits = 2*hcore_EMBED_mo.shape[0]

one_body_terms_std = np.zeros((n_qubits, n_qubits))
two_body_terms_std = np.zeros((n_qubits, n_qubits, n_qubits, n_qubits))

for p in range(n_qubits//2):
    for q in range(n_qubits//2):
        
        one_body_terms_std[2*p, 2*q] = hcore_standard_mo[p,q] # spin UP
        one_body_terms_std[(2*p + 1), (2*q +1)] = hcore_standard_mo[p,q] # spin DOWN
        
        # continue 2-body terms
        for r in range(n_qubits//2):
            for s in range(n_qubits//2):
                                
                ### SAME spin                
                two_body_terms_std[2*p, 2*q , 2*r, 2*s] = pyscf_mo_ints_std[p,q,r,s] # up up up up
                two_body_terms_std[(2*p+1), (2*q +1) , (2*r + 1), (2*s +1)] = pyscf_mo_ints_std[p,q,r,s] # down down down down
                
                ### mixed spin                
                two_body_terms_std[2*p, 2*q , (2*r + 1), (2*s +1)] = pyscf_mo_ints_std[p,q,r,s] # up up down down
                two_body_terms_std[(2*p+1), (2*q +1) , 2*r, 2*s] = pyscf_mo_ints_std[p,q,r,s] # down down up up 
               
                
                
                
### remove vanishing terms
EQ_Tolerance=1e-8
one_body_terms_std[np.abs(one_body_terms_std)<EQ_Tolerance]=0
two_body_terms_std[np.abs(two_body_terms_std)<EQ_Tolerance]=0

In [ ]:
H_fermionic = FermionOperator((),  full_system_mol.energy_nuc())

# one body terms
for p in range(one_body_terms_std.shape[0]):
    for q in range(one_body_terms_std.shape[0]):
        
        H_fermionic += one_body_terms_std[p,q] * FermionOperator(((p, 1), (q, 0)))
        
        # two body terms
        for r in range(two_body_terms_std.shape[0]):
            for s in range(two_body_terms_std.shape[0]):
                
                ######## physist notation
                ## (requires:
                ##           two_body_terms = two_body_terms.transpose(0,2,3,1) before loop starts!
                ##)
#                 H_qubit += 0.5*two_body_terms[p,q,r,s] * FermionOperator(((p, 1), (q, 1), (r,0), (s, 0)))
                
                ######## chemist notation
                H_fermionic += 0.5*two_body_terms_std[p,q,r,s] * FermionOperator(((p, 1), (r, 1), (s,0), (q, 0)))
H_fermionic

In [ ]:
H_qubit_JW = jordan_wigner(H_fermionic)
H_qubit_JW

In [ ]:
len(list(H_qubit_JW))

In [ ]:
H_JW_mat = get_sparse_operator(H_qubit_JW)

In [ ]:
E_WF_emb = (eigvecs_EMBED.conj().T @ H_JW_mat @ eigvecs_EMBED).real
E_WF_emb

In [ ]:
H_core_std = FermionOperator((),  full_system_mol.energy_nuc())

# one body terms
for p in range(one_body_terms_std.shape[0]):
    for q in range(one_body_terms_std.shape[0]):
        
        H_core_std += one_body_terms_std[p,q] * FermionOperator(((p, 1), (q, 0)))

In [ ]:
H_embed_core = FermionOperator((),  full_system_mol.energy_nuc())

# one body terms
for p in range(one_body_terms.shape[0]):
    for q in range(one_body_terms.shape[0]):
        
        H_embed_core += one_body_terms[p,q] * FermionOperator(((p, 1), (q, 0)))

In [ ]:
H_core_std_JW = jordan_wigner(H_core_std)
H_core_std_JW_mat = get_sparse_operator(H_core_std_JW)

H_embed_core_std_JW = jordan_wigner(H_embed_core)
H_embed_core_std_JW_mat = get_sparse_operator(H_embed_core_std_JW)

In [ ]:
WF_correction = eigvecs_EMBED.conj().T @ (H_core_std_JW_mat - H_embed_core_std_JW_mat) @ eigvecs_EMBED

In [ ]:
E_high_lvl_DFT = E_WF_emb[0][0] + E_env + two_e_cross + full_system_scf.energy_nuc() + correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

In [ ]:
embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

embedded_cc_obj.nocc = EMBEDDED_full_system_scf_HF.mol.nelectron // 2
embedded_cc_obj.nmo = EMBEDDED_full_system_scf_HF.mo_energy.size
# NEED to redefine embedded h_core again!
embedded_cc_obj._scf.get_hcore = lambda *args: v_emb + h_core
e_cc, t1, t2 = embedded_cc_obj.kernel()

In [ ]:
embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

# NEED to redefine embedded h_core again!
embedded_cc_obj._scf.get_hcore = lambda *args: v_emb + h_core
e_cc, t1, t2 = embedded_cc_obj.kernel()

In [ ]:
np.isclose(embedded_cc_obj.e_hf,e_hf_check) # check HF and CC_hf match!

In [ ]:
C_wf = embedded_cc_obj.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0] # take occupied C_coeff

dm_wf = 2* C_wf @ C_wf.conj().T # THIS is embedded dm matrix from cc calc!
np.isclose(np.trace(dm_wf@S), 2*len(active_ao_ind))

In [ ]:
# check everything makes sense:
embedded_cc_obj.e_tot == EMBEDDED_full_system_scf_HF.energy_tot(dm=dm_wf) + e_cc

In [ ]:
WF_correction = (matrix_dot(v_emb, dm_wf - dm_active))
WF_correction

# Finally
3. Run everything again, using optimized embedded density matrix using standard system obj!

In [ ]:
full_system_mol_HF_normal_obj = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_HF_normal_obj.build()

# redefine number of electrons in system
full_system_mol_HF_normal_obj.nelectron = 2*len(active_ao_ind)

full_system_scf_HF_normal_obj = scf.RHF(full_system_mol_HF_normal_obj)
full_system_scf_HF_normal_obj.verbose=1
full_system_scf_HF_normal_obj.max_memory= 8_000
full_system_scf_HF_normal_obj.conv_tol = 1e-6

# do NOT redefine h_core to be embedded

# Also don't run kernel, as we have op density matrix

In [ ]:
h1e=full_system_scf_HF_normal_obj.get_hcore()
s1e=full_system_scf_HF_normal_obj.get_ovlp()
vhf=full_system_scf_HF_normal_obj.get_veff(dm=dm_wf) # <-- gamma_A_embedded!


fock = full_system_scf_HF_normal_obj.get_fock(h1e,
                                              s1e,
                                              vhf,
                                              dm_wf)  # <-- gamma_A_embedded!


mo_energy, mo_coeff = full_system_scf_HF_normal_obj.eig(fock, s1e)
mo_occ = full_system_scf_HF_normal_obj.get_occ(mo_energy, mo_coeff)
e_tot = full_system_scf_HF_normal_obj.energy_tot(dm_wf, # <-- gamma_A_embedded!
                                                          h1e, vhf)


# full_system_scf_HF_normal_obj.mo_coeff = mo_coeff
full_system_scf_HF_normal_obj.mo_coeff =  embedded_cc_obj.mo_coeff#[:,embedded_wf.mo_occ>0] # take occupied C_coeff

# full_system_scf_HF_normal_obj.mo_energy = mo_energy
full_system_scf_HF_normal_obj.mo_occ = mo_occ
full_system_scf_HF_normal_obj.e_tot = e_tot
e_tot

In [ ]:
full_system_scf_HF_normal_obj.energy_tot(dm=dm_wf)

In [ ]:
# ## checking if mo_energy makes sense (as overwriting mo_coeff)
# S_neg1 = sp.linalg.inv(s1e)
# C_neg1 = sp.linalg.inv(EMBEDDED_full_system_scf.mo_coeff)

# e = C_neg1 @ S_neg1 @ fock @ EMBEDDED_full_system_scf.mo_coeff

# u, E, V = np.linalg.svd(e)

# np.allclose(np.abs(mo_energy[mo_occ>0]), np.abs(E[mo_occ>0]))

In [ ]:
cc_standard_obj = cc.CCSD(full_system_scf_HF_normal_obj)


# embedded_cc_obj.nocc = EMBEDDED_full_system_scf_HF.mol.nelectron // 2
# embedded_cc_obj.nmo = EMBEDDED_full_system_scf_HF.mo_energy.size

e_cc, t1_standard, t2_standard = cc_standard_obj.kernel()

np.isclose(cc_standard_obj.e_hf, e_tot)

In [ ]:
C_wf = embedded_cc_obj.mo_coeff # take embedded gamma and calc energy WITHOUT embedded H
eris = cc_standard_obj.ao2mo(mo_coeff=C_wf)

# calculate coupled cluster correction to e_hf with same t1 and t2 as found using embedded Hcore
e_cc_standard_with_embedded_Density = cc_standard_obj.energy(t1, t2, eris)

# calculate coupled cluster correction to e_hf with NEW t1 and t2 found using WITHOUT embedded Hcore
# BUT use C matrix from embedded Hcore calculation!
# e_cc_standard_with_embedded_Density = cc_standard_obj.energy(t1_standard, t2_standard, eris)


e_cc_standard_with_embedded_Density

In [ ]:
C_wf_opt = cc_standard_obj.mo_coeff[:,cc_standard_obj.mo_occ>0] # take occupied C_coeff

dm_wf_opt = 2* C_wf_opt @ C_wf_opt.conj().T

print('checking no. of electrons is correct:', np.isclose(np.trace(dm_wf_opt@s1e), 2*len(active_ao_ind)))
print()
print('checking CC dm matrix matches embedded dm matrix:', np.allclose(dm_wf_opt, dm_wf))

In [ ]:
E_WF_standard_obj = (cc_standard_obj.e_tot 
                     + E_env + two_e_cross + WF_correction)
# E__WF # <-- energy from embedded WF calc

np.abs(E_WF_standard_obj-my_fci.e_tot)

In [ ]:
E_WF_standard_obj = (cc_standard_obj.e_hf + e_cc_standard_with_embedded_Density
                     + E_env + two_e_cross + WF_correction)
# E__WF # <-- energy from embedded WF calc

np.abs(E_WF_standard_obj-my_fci.e_tot)

In [ ]:
E_WF_standard_obj = (cc_standard_obj.e_corr + e_tot 
                     + E_env + two_e_cross + WF_correction)
# E__WF # <-- energy from embedded WF calc

np.abs(E_WF_standard_obj-my_fci.e_tot)

In [ ]:
dsaas

# OTHER WF way

1. Run higher level method using dm from RKS
2. don't try use HF to find other embedded gamma_A

In [ ]:
alt_full_system_mol_HF_normal_obj = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
alt_full_system_mol_HF_normal_obj.build()

# redefine number of electrons in system
alt_full_system_mol_HF_normal_obj.nelectron = 2*len(active_ao_ind)

alt_full_system_scf_HF_normal_obj = scf.RHF(alt_full_system_mol_HF_normal_obj)
alt_full_system_scf_HF_normal_obj.verbose=1
alt_full_system_scf_HF_normal_obj.max_memory= 8_000
alt_full_system_scf_HF_normal_obj.conv_tol = 1e-6

# do NOT redefine h_core to be embedded

# Also don't run kernel, as we have op density matrix



h1e=alt_full_system_scf_HF_normal_obj.get_hcore()
s1e=alt_full_system_scf_HF_normal_obj.get_ovlp()
vhf=alt_full_system_scf_HF_normal_obj.get_veff(dm=density_emb) # <-- gamma_A_embedded from DFT calc!


fock = alt_full_system_scf_HF_normal_obj.get_fock(h1e,
                                              s1e,
                                              vhf,
                                              density_emb)  # <-- gamma_A_embedded!



mo_energy, mo_coeff = alt_full_system_scf_HF_normal_obj.eig(fock, s1e)
mo_occ = alt_full_system_scf_HF_normal_obj.get_occ(mo_energy, mo_coeff)
e_tot = alt_full_system_scf_HF_normal_obj.energy_tot(density_emb, # <-- gamma_A_embedded!
                                                          h1e, vhf)



###########################

# something going WRONG HERE
# should set:
#             alt_full_system_scf_HF_normal_obj.mo_coeff = EMBEDDED_full_system_scf.mo_coeff
# rather than updated version from eig

alt_full_system_scf_HF_normal_obj.mo_coeff = mo_coeff

# alt_full_system_scf_HF_normal_obj.mo_coeff =  EMBEDDED_full_system_scf.mo_coeff

# C_pseduo_inv = np.linalg.pinv(EMBEDDED_full_system_scf.mo_coeff)
# C_pseudo = sp.linalg.inv(C_pseduo_inv)
# alt_full_system_scf_HF_normal_obj.mo_coeff = C_pseudo

##############################



alt_full_system_scf_HF_normal_obj.mo_energy = mo_energy
alt_full_system_scf_HF_normal_obj.mo_occ = mo_occ
alt_full_system_scf_HF_normal_obj.e_tot = e_tot
e_tot

In [ ]:
alt_full_system_scf_HF_normal_obj.energy_tot(dm=density_emb)

In [ ]:
np.allclose(alt_full_system_scf_HF_normal_obj.make_rdm1(
        EMBEDDED_full_system_scf.mo_coeff, EMBEDDED_full_system_scf.mo_occ)
            , density_emb)

In [ ]:
alt_cc_standard_obj = cc.CCSD(alt_full_system_scf_HF_normal_obj)
# cc_standard_obj.diis = False

# embedded_cc_obj.nocc = EMBEDDED_full_system_scf_HF.mol.nelectron // 2
# embedded_cc_obj.nmo = EMBEDDED_full_system_scf_HF.mo_energy.size

e_cc, t1_standard, t2_standard = alt_cc_standard_obj.kernel()

np.isclose(alt_cc_standard_obj.e_hf, e_tot)

In [ ]:
E_WF_standard_obj = (alt_cc_standard_obj.e_corr + e_tot
                     + E_env + two_e_cross + WF_correction)
# E__WF # <-- energy from embedded WF calc

np.abs(E_WF_standard_obj-my_fci.e_tot)

In [ ]:
C_wf_opt = alt_cc_standard_obj.mo_coeff[:,cc_standard_obj.mo_occ>0] # take occupied C_coeff

dm_wf_opt = 2* C_wf_opt @ C_wf_opt.conj().T

print('checking no. of electrons is correct:', np.isclose(np.trace(dm_wf_opt@s1e), 2*len(active_ao_ind)))
print()
print('checking CC dm matrix matches embedded dm matrix:', np.allclose(dm_wf_opt, density_emb))

In [ ]:
# if CC dm matrix is not matching, this is due to NOT setting:
#     alt_full_system_scf_HF_normal_obj.mo_coeff = EMBEDDED_full_system_scf.mo_coeff

In [ ]:
alt_cc_standard_obj.e_hf

In [ ]:
cc_standard_obj.e_hf + e_cc_standard_with_embedded_Density

# QC work

$$H_{fermionic} = h_{nuc} + \sum_{p, q} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p, q ,r ,s} h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$

where:

- $$h_{pq} = \int d{\vec{x}} \phi_{p}^{*}({\vec{x}}) \bigg( - \frac{\nabla^{2}_{{\vec{r}}}}{2} - \sum_{I} \frac{Z_{I}}{|{\vec{r}} - {\vec{R}_{I}} |} \bigg) \phi_{q}({\vec{x}})$$


- $$h_{pqrs} = \int d{\vec{x}}_{1} d{\vec{x}}_{2} \frac{\phi_{p}^{*}({\vec{x}}_{1}) \phi_{q}^{*}({\vec{x}}_{2}) \phi_{s}({\vec{x}}_{1}) \phi_{r}({\vec{x}}_{2})}{|{\vec{r}_{1}} - {\vec{r}}_{2}|}$$ 


- $$h_{nuc} = \frac{1}{2} \sum_{I \neq J} \frac{Z_{I} Z_{J}}{| {\vec{R}}_{I} - {\vec{R}}_{J}|}$$

In [ ]:
#2D array for one-body Hamiltonian (H_core) in the MO representation

Hcore_normal =  full_system_scf_HF_normal_obj.get_hcore()
# canonical_orbitals  = full_system_scf_HF_normal_obj.mo_coeff # HF calc WITH embedded density matrix, but standard molecule obj!
canonical_orbitals  = full_system_scf_HF_normal_obj.mo_coeff[:,full_system_scf_HF_normal_obj.mo_occ>0]

one_body_integrals = canonical_orbitals.conj().T @ Hcore_normal @ canonical_orbitals
one_body_integrals.shape

In [ ]:
# A 4-dimension array for electron repulsion integrals in the MO
# representation.  The integrals are computed as
# h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
from pyscf import ao2mo


eri = ao2mo.kernel(full_system_mol_HF_normal_obj, canonical_orbitals)
n_orbitals = canonical_orbitals.shape[1]
eri = ao2mo.restore(1, # no permutation symmetry
                      eri, 
                    n_orbitals)


two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order='C')
two_body_integrals.shape

In [ ]:
def spinorb_from_spatial(one_body_integrals, two_body_integrals):
    EQ_TOLERANCE=1e-8
    n_qubits = 2 * one_body_integrals.shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros(
        (n_qubits, n_qubits, n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[p, q]
            one_body_coefficients[2 * p + 1, 2 * q +
                                  1] = one_body_integrals[p, q]
            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r + 1, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q + 1, 2 * r +
                                          1, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

    # Truncate.
    one_body_coefficients[
        np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.
    two_body_coefficients[
        np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.

    return one_body_coefficients, two_body_coefficients


In [ ]:
one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals,
                                                                    two_body_integrals)

In [ ]:
from openfermion.ops.representations import InteractionOperator

constant = full_system_scf.energy_nuc() 

molecular_hamiltonian = InteractionOperator(constant,
                                            one_body_coefficients,
                                            1 / 2 * two_body_coefficients)

n_qubits = two_body_coefficients.shape[0]

from openfermion.transforms import jordan_wigner
Qubit_Hamiltonian = jordan_wigner(molecular_hamiltonian)

In [ ]:
len(list(Qubit_Hamiltonian))

In [ ]:
from openfermion import get_sparse_operator
H_matrix = get_sparse_operator(Qubit_Hamiltonian)

In [ ]:
from scipy.linalg import eigh
eig_val, eig_vec = eigh(H_matrix.todense())


In [ ]:
min(eig_val)

In [ ]:
E_VQE = min(eig_val)

In [ ]:
E_VQE_in_DFT = (E_VQE + 
                E_env + two_e_cross + WF_correction)
E_VQE_in_DFT

In [ ]:
np.abs(E_VQE_in_DFT-my_fci.e_tot)

In [ ]:
TODO everything below here

# incorrect code below!

In [ ]:
#2D array for one-body Hamiltonian (H_core) in the MO representation

EMBEDDED_Hcore =  full_system_scf_HIGH_LEVEL_HF.get_hcore()
canonical_orbitals  = full_system_scf_HIGH_LEVEL_HF.mo_coeff 

one_body_integrals = canonical_orbitals.conj().T @ EMBEDDED_Hcore @ canonical_orbitals
one_body_integrals.shape

In [ ]:
# A 4-dimension array for electron repulsion integrals in the MO
# representation.  The integrals are computed as
# h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
from pyscf import ao2mo

# eri = ao2mo.kernel(EMBEDDED_HF_full_system_scf, canonical_orbitals)

eri = ao2mo.kernel(full_system_mol_HIGH_LEVEL_WF_mol, canonical_orbitals)
n_orbitals = canonical_orbitals.shape[1]
eri = ao2mo.restore(1, # no permutation symmetry
                                eri, n_orbitals)


two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order='C')
two_body_integrals.shape

In [ ]:
def spinorb_from_spatial(one_body_integrals, two_body_integrals):
    EQ_TOLERANCE=1e-8
    n_qubits = 2 * one_body_integrals.shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros(
        (n_qubits, n_qubits, n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[p, q]
            one_body_coefficients[2 * p + 1, 2 * q +
                                  1] = one_body_integrals[p, q]
            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r + 1, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q + 1, 2 * r +
                                          1, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

    # Truncate.
    one_body_coefficients[
        np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.
    two_body_coefficients[
        np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.

    return one_body_coefficients, two_body_coefficients


In [ ]:
one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals,
                                                                    two_body_integrals)

In [ ]:
from openfermion.ops.representations import InteractionOperator

constant = full_system_scf.energy_nuc() 

molecular_hamiltonian = InteractionOperator(constant,
                                            one_body_coefficients,
                                            1 / 2 * two_body_coefficients)

n_qubits = two_body_coefficients.shape[0]

from openfermion.transforms import jordan_wigner
Qubit_Hamiltonian = jordan_wigner(molecular_hamiltonian)

In [ ]:
len(list(Qubit_Hamiltonian))

In [ ]:
from openfermion import get_sparse_operator
H_matrix = get_sparse_operator(Qubit_Hamiltonian)

In [ ]:
print(embedded_wf.e_tot)
print(full_system_scf_HIGH_LEVEL_HF.e_tot + correlation_energy)
# np.isclose(embedded_wf.e_tot,  )

In [ ]:
## not sure if this
shift = full_system_mol.nbas - n_env_mos
frozen_orbitals = [i for i in range(shift, full_system_mol.nbas)]

## OR
frozen_orbitals = enviro_ind_occ

In [ ]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_EMBEDDED_HF.build()
full_system_mol_EMBEDDED_HF.nelectron = int(np.around(np.trace(dm_active@S)))

EMBEDDED_HF_full_system_scf = scf.RHF(full_system_mol_EMBEDDED_HF)
EMBEDDED_HF_full_system_scf.verbose=1
EMBEDDED_HF_full_system_scf.max_memory= 8_000
EMBEDDED_HF_full_system_scf.conv_tol = 1e-6

h_core = EMBEDDED_HF_full_system_scf.get_hcore()
EMBEDDED_HF_full_system_scf.get_hcore = lambda *args: v_emb + h_core

EMBEDDED_HF_full_system_scf.kernel() # <-- this would run away from psi embedded

dm_act_EMBEDDED = EMBEDDED_HF_full_system_scf.make_rdm1()
# dm_act_EMBEDDED = 2 * act_orbitals @ act_orbitals.T

h1e = EMBEDDED_HF_full_system_scf.get_hcore()
s1e = EMBEDDED_HF_full_system_scf.get_ovlp()
vhf= EMBEDDED_HF_full_system_scf.get_veff(dm=dm_act_EMBEDDED)

fock = EMBEDDED_HF_full_system_scf.get_fock(h1e, s1e, vhf, dm_act_EMBEDDED)  # = h1e + vhf, no DIIS
mo_energy, mo_coeff = EMBEDDED_HF_full_system_scf.eig(fock, s1e)
mo_occ = EMBEDDED_HF_full_system_scf.get_occ(mo_energy, mo_coeff)
e_tot = EMBEDDED_HF_full_system_scf.energy_tot(dm_act_EMBEDDED, h1e, vhf)


EMBEDDED_HF_full_system_scf.mo_energy = mo_energy
EMBEDDED_HF_full_system_scf.mo_coeff = mo_coeff
EMBEDDED_HF_full_system_scf.mo_occ = mo_occ
EMBEDDED_HF_full_system_scf.e_tot = e_tot
EMBEDDED_HF_full_system_scf.converged = True
e_tot

In [ ]:
np.trace(dm_act_EMBEDDED@S)

In [ ]:
embedded_wf = cc.CCSD(EMBEDDED_HF_full_system_scf).set(frozen = frozen_orbitals).run()
correlation_energy = embedded_wf.e_corr
correlation_energy

In [ ]:
# uses new embedded density matrix!
e_act_emb_NEW = matrix_dot(dm_act_EMBEDDED, h_core + 0.5 * J_emb - 0.25 * K_emb)
# e_act_emb_NEW = matrix_dot(dm_act_EMBEDDED, h_core + 0.5 * vhf)
e_act_emb_NEW

In [ ]:
E_corrected = (e_act_emb_NEW + E_env + two_e_cross + 
               full_system_scf.energy_nuc() + correction + correlation_energy) 
E_corrected

In [ ]:
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= 8_000
HF_scf.conv_tol = 1e-6
HF_scf.xc = low_level_xc_functional_or_HF
HF_scf.kernel()

cisolver = fci.FCI(HF_scf)
print('E(UHF-FCI) = %.12f' % cisolver.kernel()[0])

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

In [ ]:
abs(myci.e_tot - e_mf_emb)

In [ ]:
abs(myci.e_tot - E_corrected)

In [ ]:
abs(myci.e_tot - (e_mf_emb + correlation_energy )) # not (what PsiEmbed does)

In [ ]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()
full_system_mol.verbose = 1
full_system_mol.max_memory = 8_000 #MB
full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = high_level_xc_functional # <-- higher level DFT calc of supersystem!!!
full_system_scf.kernel()

full_system_scf.e_tot

In [ ]:
print(abs(full_system_scf.e_tot - e_mf_emb))
print(abs(full_system_scf.e_tot - E_corrected))
print(abs(full_system_scf.e_tot - (e_mf_emb + correlation_energy )))

In [ ]:
fock = EMBEDDED_HF_full_system_scf.get_fock(h1e, s1e, vhf, dm_act_EMBEDDED)  # = h1e + vhf, no DIIS
mo_energy, mo_coeff = EMBEDDED_HF_full_system_scf.eig(fock, s1e)

# eig_val, eig_vec = eigh(fock)
# eig_val

In [ ]:
# h_core + 0.5 * vhf

# QC calc

In [ ]:
# #2D array for one-body Hamiltonian (H_core) in the MO representation

# EMBEDDED_Hcore =  EMBEDDED_HF_full_system_scf.get_hcore() # contains v_embedding!
# canonical_orbitals  = EMBEDDED_HF_full_system_scf.mo_coeff 


# one_body_integrals = canonical_orbitals.conj().T @ EMBEDDED_Hcore @ canonical_orbitals
# one_body_integrals.shape

In [ ]:
#2D array for one-body Hamiltonian (H_core) in the MO representation

EMBEDDED_Hcore =  h_core # does NOT!!!!!!! contain v_embedding!
canonical_orbitals  = EMBEDDED_HF_full_system_scf.mo_coeff # <-- this is optimized wavefunction WITH embedded vhf


one_body_integrals = canonical_orbitals.conj().T @ EMBEDDED_Hcore @ canonical_orbitals
one_body_integrals.shape

In [ ]:
# A 4-dimension array for electron repulsion integrals in the MO
# representation.  The integrals are computed as
# h[p,q,r,s]=\int \phi_p(x)* \phi_q(y)* V_{elec-elec} \phi_r(y) \phi_s(x) dxdy
from pyscf import ao2mo

# eri = ao2mo.kernel(EMBEDDED_HF_full_system_scf, canonical_orbitals)

eri = ao2mo.kernel(full_system_mol_EMBEDDED_HF, canonical_orbitals)
n_orbitals = canonical_orbitals.shape[1]
eri = ao2mo.restore(1, # no permutation symmetry
                                eri, n_orbitals)


two_body_integrals = np.asarray(eri.transpose(0, 2, 3, 1), order='C')
two_body_integrals.shape

In [ ]:
shift = full_system_mol.nbas - n_env_mos
frozen_orbitals = [i for i in range(shift, full_system_mol.nbas)]

# frozen_orbitals = enviro_ao_ind
frozen_orbitals

In [ ]:
# https://github.com/quantumlib/OpenFermion/blob/84d169b529a27a0cfd1cc41073d2665f1fcf0312/src/openfermion/ops/representations/interaction_operator.py#L210

## define active space!

active_indices = [i for i in range(0, shift)]
occupied_indices = np.where(EMBEDDED_HF_full_system_scf.mo_occ>0)[0]

# active_indices = active_ao_ind

core_adjustment = 0.0
for i in occupied_indices:
    core_adjustment += 2 * one_body_integrals[i, i]
    for j in occupied_indices:
        core_adjustment += (2 * two_body_integrals[i, j, j, i] -
                          two_body_integrals[i, j, i, j])

# Modified one electron integrals
one_body_integrals_new = np.copy(one_body_integrals)
for u in active_indices:
    for v in active_indices:
        for i in occupied_indices:
            one_body_integrals_new[u, v] += (
                2 * two_body_integrals[i, u, v, i] -
                two_body_integrals[i, u, i, v])

# Restrict integral ranges and change M appropriately

one_body_integrals_NEW = one_body_integrals_new[np.ix_(active_indices, active_indices)]

two_body_integrals_NEW = two_body_integrals[np.ix_(active_indices, active_indices,
                             active_indices, active_indices)]

In [ ]:
def spinorb_from_spatial(one_body_integrals, two_body_integrals):
    EQ_TOLERANCE=1e-8
    n_qubits = 2 * one_body_integrals.shape[0]

    # Initialize Hamiltonian coefficients.
    one_body_coefficients = np.zeros((n_qubits, n_qubits))
    two_body_coefficients = np.zeros(
        (n_qubits, n_qubits, n_qubits, n_qubits))
    # Loop through integrals.
    for p in range(n_qubits // 2):
        for q in range(n_qubits // 2):

            # Populate 1-body coefficients. Require p and q have same spin.
            one_body_coefficients[2 * p, 2 * q] = one_body_integrals[p, q]
            one_body_coefficients[2 * p + 1, 2 * q +
                                  1] = one_body_integrals[p, q]
            # Continue looping to prepare 2-body coefficients.
            for r in range(n_qubits // 2):
                for s in range(n_qubits // 2):

                    # Mixed spin
                    two_body_coefficients[2 * p, 2 * q + 1, 2 * r + 1, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q, 2 * r, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

                    # Same spin
                    two_body_coefficients[2 * p, 2 * q, 2 * r, 2 *
                                          s] = (two_body_integrals[p, q, r, s])
                    two_body_coefficients[2 * p + 1, 2 * q + 1, 2 * r +
                                          1, 2 * s +
                                          1] = (two_body_integrals[p, q, r, s])

    # Truncate.
    one_body_coefficients[
        np.absolute(one_body_coefficients) < EQ_TOLERANCE] = 0.
    two_body_coefficients[
        np.absolute(two_body_coefficients) < EQ_TOLERANCE] = 0.

    return one_body_coefficients, two_body_coefficients

In [ ]:
one_body_coefficients, two_body_coefficients = spinorb_from_spatial(one_body_integrals_NEW, two_body_integrals_NEW)

In [ ]:
# one_body_integrals_NEW =one_body_integrals
# two_body_integrals_NEW =two_body_integrals

In [ ]:
from openfermion.ops.representations import InteractionOperator

constant = full_system_scf.energy_nuc() + core_adjustment

molecular_hamiltonian = InteractionOperator(constant,
                                            one_body_integrals_NEW,
                                            1 / 2 * two_body_integrals_NEW)

n_qubits = one_body_integrals_NEW.shape[0]

In [ ]:
from openfermion.transforms import jordan_wigner
Qubit_Hamiltonian = jordan_wigner(molecular_hamiltonian)

In [ ]:
len(list(Qubit_Hamiltonian))

In [ ]:
from functools import reduce

H_trimmed = [op for op in Qubit_Hamiltonian if list(op.terms.values())[0]>1e-5]
# H_trimmed_op = reduce(lambda x,y: x+y, H_trimmed_op)
len(H_trimmed)

In [ ]:
from openfermion import QubitOperator
H_trimmed_op = QubitOperator()
for term in H_trimmed:
    H_trimmed_op+=term

In [ ]:
# H_matrix = get_sparse_operator(H_trimmed_op)

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
Qubit_Hamiltonian

In [ ]:
from openfermion import get_sparse_operator
H_matrix = get_sparse_operator(Qubit_Hamiltonian)

In [ ]:
Qubit_Hamiltonian

In [ ]:
from scipy.linalg import eigh

eig_val, eig_vec = eigh(H_matrix.todense())

In [ ]:
min(eig_val)

In [ ]:
idx = eig_val.argsort()   
eigenValues = eig_val[idx]
eigenVectors = eig_vec[:,idx]

In [ ]:
WF_ground_state = eigenVectors[:, 0] 

dm_act_EMBEDDED_from_WF = 2 * np.outer(WF_ground_state, WF_ground_state)
# np.trace(dm_act_EMBEDDED_from_WF@s1e)

In [ ]:
dm_act_EMBEDDED_from_WF.shape

In [ ]:
E_WF_in_DFT = (min(eig_val) + E_env + two_e_cross + correction) 
E_WF_in_DFT

In [ ]:
E_final = (eigenVectors[:, 0].conj().T  @ H_matrix.toarray() @  eigenVectors[:, 0]  +
           E_env + two_e_cross + correction)
E_final

In [ ]:
print(abs(myci.e_tot - E_final))

1. Global calc


2. localize orbitals
    - SPADE
    - PM
    
3. define active system and enirvo
4. Define V_emb
5. Find gamma_active_EMBEDDED
6. Calculate Energy of A with gamma_active_EMBEDDED